# Lifecycle Reports (BMC Discovery)

This notebook consolidates the database, operating system, and software lifecycle reports into a single workflow.

## Requirements

In [ ]:
# %pip install -q tideway pandas pyyaml

import sys
from pathlib import Path

import pandas as pd
import yaml
from IPython.display import display


## Select Appliance (optional)

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection
#=APPLIANCE_INDEX = 1

## Configuration (from config.yaml)

In [ ]:
# Locate the repository root (directory containing config.yaml)
def _find_repo_root(start: Path) -> Path:
    for candidate in (start,) + tuple(start.parents):
        if (candidate / 'config.yaml').exists():
            return candidate
    raise FileNotFoundError('Unable to locate config.yaml from current working directory')

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
with config_path.open('r') as fh:
    cfg = yaml.safe_load(fh) or {}

apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target" entry')

# Resolve API token (inline or from a token_file reference)
token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with tf_path.open('r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Appliance     :', (selected or {}).get('name', '(single)'))
print('Base URL      :', target)
print('API Version   :', API_VERSION)
print('Verify SSL    :', VERIFY_SSL)
print('Output folder :', output_dir)

try:
    import tideway  # type: ignore
except ImportError:
    import subprocess
    print('Installing tideway via pip...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tideway'])
    import tideway  # type: ignore

API_VERSION_NUM = API_VERSION.lstrip('v')
app = tideway.appliance(target, token, api_version=API_VERSION_NUM, ssl_verify=VERIFY_SSL)
twsearch = app.data()

try:
    about = app.api_about
    print('Appliance reachable:', getattr(about, 'status_code', 'unknown'))
except Exception as exc:
    print('Warning: failed to contact appliance /api/about:', exc)


## Session and helpers

In [ ]:
DEFAULT_PAGE_SIZE = 500


def _normalize_result(raw):
    if raw is None:
        return [], []
    if hasattr(raw, 'json'):
        try:
            raw = raw.json()
        except Exception:
            return [], []
    if isinstance(raw, dict):
        headings = raw.get('headings') or raw.get('columns') or []
        rows = raw.get('results') or raw.get('rows') or raw.get('data') or []
        if isinstance(rows, dict):
            return _normalize_result(rows)
        if isinstance(rows, list):
            if rows and isinstance(rows[0], list) and headings:
                records = [dict(zip(headings, row)) for row in rows]
            else:
                records = rows
        else:
            records = []
        return headings, records
    if isinstance(raw, list):
        if not raw:
            return [], []
        first = raw[0]
        if isinstance(first, dict) and 'headings' in first and 'results' in first:
            return _normalize_result(first)
        if isinstance(first, list):
            headings = first
            records = []
            for item in raw[1:]:
                if isinstance(item, dict):
                    records.append({key: item.get(key) for key in headings})
                elif isinstance(item, list):
                    records.append(dict(zip(headings, item)))
            return headings, records
        if isinstance(first, dict):
            records = raw
            headings = sorted({key for row in records for key in getattr(row, 'keys', lambda: [])()})
            return headings, records
    return [], []


def post_search(query: str, *, limit: int | None = None, page_size: int = DEFAULT_PAGE_SIZE):
    fetch_all = (limit == 0)
    limit_arg = None
    if fetch_all:
        limit_arg = 0  # tideway treats 0 as "all results"
    elif limit is not None:
        try:
            limit_arg = max(int(limit), 0)
        except Exception:
            limit_arg = None
    else:
        per_page = page_size or DEFAULT_PAGE_SIZE
        try:
            per_page = max(int(per_page), 0)
        except Exception:
            per_page = DEFAULT_PAGE_SIZE
        limit_arg = per_page or None

    kwargs = {'format': 'object'}
    if limit_arg is not None:
        kwargs['limit'] = limit_arg

    try:
        raw = twsearch.search({'query': query}, **kwargs)
    except Exception as exc:
        print('Error executing search:', exc)
        return {'headings': [], 'results': []}

    headings, records = _normalize_result(raw)
    if not fetch_all and limit_arg and limit_arg > 0 and len(records) > limit_arg:
        records = records[:limit_arg]
    if not headings and records and isinstance(records[0], dict):
        headings = sorted({key for row in records for key in row.keys()})
    return {'headings': headings, 'results': records}


def to_dataframe(result: dict) -> pd.DataFrame:
    headings = result.get('headings') or []
    rows = result.get('results') or []
    if not rows:
        return pd.DataFrame(columns=headings)
    first = rows[0]
    if isinstance(first, dict):
        df = pd.DataFrame(rows)
        if headings:
            for col in headings:
                if col not in df.columns:
                    df[col] = pd.NA
            df = df[headings]
        return df
    return pd.DataFrame(rows, columns=headings if headings else None)


## Report definitions

In [ ]:
REPORTS = [
    {
        'key': 'database',
        'title': 'Database Lifecycle Report',
        'filename': 'db_lifecycle.csv',
        'query': """search Pattern
where 'Relational Database Management Systems' in categories
traverse Pattern:Maintainer:Element:SoftwareInstance
    where #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
       or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
       or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
show
    type as "SoftwareInstance.type",
    product_version as "SoftwareInstance.product_version",
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
        and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date, '%Y-%m-%d')) as 'End of Life',
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
        and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date, '%Y-%m-%d')) as 'End of Support',
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
        and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date, '%Y-%m-%d')) as 'End of Ext Support',
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
        and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date < currentTime() and 'EOES Exceeded')
        or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
            and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date < currentTime() and 'EOS Exceeded')
            or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
                and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date < currentTime() and 'EOL Exceeded')
                or (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date < currentTime() + 182 * 864000000000 and 'EOL less than 6 months away')
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date < currentTime() + 182 * 864000000000 and 'EOS less than 6 months away')
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date < currentTime() + 182 * 864000000000 and 'EOES less than 6 months away'))
                or (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date and 'EOL more than 6 months away'
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date and 'EOS more than 6 months away'
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date and 'EOES more than 6 months away')) as 'Lifecycle Risk'""",
    },
    {
        'key': 'os',
        'title': 'OS Lifecycle Report',
        'filename': 'os_lifecycle.csv',
        'query': """search Host
where #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date
    or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date
        or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date
show
    name,
    (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date
        and formatTime(#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date, '%Y-%m-%d')) as 'End of Life',
    (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date
        and formatTime(#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date, '%Y-%m-%d')) as 'End of Support',
    (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date
        and formatTime(#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date, '%Y-%m-%d')) as 'End of Ext Support',
    (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date
        and (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date < currentTime()
        and 'EOES Exceeded')
        or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date
            and (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date < currentTime()
            and 'EOS Exceeded') or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date
            and (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date < currentTime()
            and 'EOL Exceeded')
            or (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date
                and (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date < currentTime() + 182 * 864000000000
                and 'EOL less than 6 months away')
                or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date
                    and (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date < currentTime() + 182 * 864000000000
                    and 'EOS less than 6 months away')
                    or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date
                        and (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date < currentTime() + 182 * 864000000000
                        and 'EOES less than 6 months away'))
                        or (#ElementWithDetail:SupportDetail:OSDetail:SupportDetail.retirement_date
                            and 'EOL more than 6 months away'
                            or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_support_date
                                and 'EOS more than 6 months away'
                                or #ElementWithDetail:SupportDetail:OSDetail:SupportDetail.end_ext_support_date
                                    and 'EOES more than 6 months away')) as 'Lifecycle Risk'""",
    },
    {
        'key': 'software',
        'title': 'Software Lifecycle Report',
        'filename': 'software_lifecycle.csv',
        'query': """search SoftwareInstance
where
#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
        or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
show
type,
product_version,
(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
    and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date, '%Y-%m-%d')) as 'End of Life',
(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
    and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date, '%Y-%m-%d')) as 'End of Support',
(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
    and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date, '%Y-%m-%d')) as 'End of Ext Support',
(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
    and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date < currentTime()
    and 'EOES Exceeded')
    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
        and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date < currentTime()
        and 'EOS Exceeded')
        or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
            and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date < currentTime()
            and 'EOL Exceeded')
            or (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
                and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date < currentTime() + 182 * 864000000000
                and 'EOL less than 6 months away')
                or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
                    and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date < currentTime() + 182 * 864000000000
                    and 'EOS less than 6 months away')
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
                        and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date < currentTime() + 182 * 864000000000
                        and 'EOES less than 6 months away'))
                        or (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
                            and 'EOL more than 6 months away'
                            or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
                                and 'EOS more than 6 months away'
                                or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
                                    and 'EOES more than 6 months away')) as 'Lifecycle Risk'""",
    },
]


## Run all lifecycle reports

In [ ]:
dataframes = {}
display_limit = 10

for report in REPORTS:
    print(f"\n=== {report['title']} ===")
    result = post_search(report['query'], limit=0)
    df = to_dataframe(result)
    row_count = len(df)
    print(f"Rows returned: {row_count}")
    if row_count > 0:
        df.insert(0, 'Discovery Instance', target)
        display(df.head(display_limit))
    output_path = output_dir / report['filename']
    output_path.parent.mkdir(parents=True, exist_ok=True)
    if row_count == 0:
        headings = result.get('headings') or []
        empty_cols = ['Discovery Instance'] + list(headings)
        pd.DataFrame(columns=empty_cols).to_csv(output_path, index=False)
        print(f'No data; created empty file at {output_path}')
    else:
        df.to_csv(output_path, index=False)
        print(f'Saved to {output_path}')
    dataframes[report['key']] = df

print('\nFinished running lifecycle reports.')
